# Data Joining and Cleaning
### Gavin Rolls

This script takes the Overture and OSM data queried from DataQuerying.ipynb as well as downloaded BRES and LSOA-level data and joins it 

## Library Imports

In [9]:
#Basics
import pandas as pd
import geopandas as gpd
import numpy as np

#Shapely
from geopy.geocoders import Nominatim
from shapely.geometry import box
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import mapping
from shapely.geometry import shape

#Plots
import matplotlib.pyplot as plt

#Warning Supression
import warnings

## UK BRES Employment Data

### London

I created a separate CSV for London employment data because it was simply easier to do outside of code (harder to filter cause it's done by borough name)

In [20]:
#Skip the first six rows because they're header information
empl_data_london = pd.read_csv('data/employment_data/lsoa_by_industry_london.csv', skiprows=7, delimiter=',')

unnamed_cols = empl_data_london.columns[empl_data_london.columns.str.contains('^Unnamed:')]
empl_data_london.drop(columns=unnamed_cols, inplace=True)

#Separate name into LSOA11CD and LSOA11NM
def split_column(value):
    #Keep Greater London stats
    if value.startswith('gor:'):
        return value, value
        #Split into name and code
    else:
        parts = value.split('lsoa2011:')[1]
        code, name = parts.split(' : ')
        return code.strip(), name.strip()
        return code, name

empl_data_london[['LSOA11CD', 'LSOA11NM']] = empl_data_london['Area'].apply(lambda x: pd.Series(split_column(x)))

print("Num Rows (LSOAs) Before Cleaning: " + str(empl_data_london.shape[0]))

#There appear to be a bunch of duplicates so I'm going to get rid of them now
empl_data_london.drop_duplicates(inplace=True)

print("Num Rows (LSOAs) After Cleaning: " + str(empl_data_london.shape[0]))

Num Rows (LSOAs) Before Cleaning: 9478
Num Rows (LSOAs) After Cleaning: 4836


### Birmingham

In [30]:
# Get rid of value set on copy of slice warning
warnings.filterwarnings('ignore')

#Skip the first six rows because they're header information
empl_data = pd.read_csv('data/employment_data/lsoa_by_industry.csv', skiprows=7, delimiter=',')

unnamed_cols = empl_data.columns[empl_data.columns.str.contains('^Unnamed:')]
empl_data.drop(columns=unnamed_cols, inplace=True)

#Separate name into LSOA11CD and LSOA11NM
def split_column(value):
    if isinstance(value, str) and 'lsoa2011:' in value:
        parts = value.split('lsoa2011:')[1]
        code, name = parts.split(' : ')
        return code.strip(), name.strip()
    else:
        return None, None

empl_data[['LSOA11CD', 'LSOA11NM']] = empl_data['Area'].apply(lambda x: pd.Series(split_column(x)))

# Drop rows not in Birmingham
empl_data.dropna(subset=['LSOA11NM'], inplace=True)
empl_data_bham = empl_data[empl_data['LSOA11NM'].str.contains('Birmingham')]

print("Num Rows (LSOAs) Before Cleaning: " + str(empl_data_bham.shape[0]))

#There appear to be a bunch of duplicates so I'm going to get rid of them now
empl_data_bham.drop_duplicates(inplace=True)

print("Num Rows (LSOAs) After Cleaning: " + str(empl_data_bham.shape[0]))

Num Rows (LSOAs) Before Cleaning: 639
Num Rows (LSOAs) After Cleaning: 639


In [32]:
# Get rid of value set on copy of slice warning
warnings.filterwarnings('ignore')

# Create Total Employment Column for London
empl_data_london[empl_data_london.columns[1:-2]] = empl_data_london[empl_data_london.columns[1:-2]].apply(pd.to_numeric, errors='coerce')
empl_data_london['total_employment'] = empl_data_london[empl_data_london.columns[1:-2]].sum(axis=1)
empl_data_london['log_total_employment'] = np.log(empl_data_london['total_employment'].replace(0, np.nan))

empl_data_london.head()

,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,08 : Other mining and quarrying,09 : Mining support service activities,10 : Manufacture of food products,...,94 : Activities of membership organisations,95 : Repair of computers and personal and household goods,96 : Other personal service activities,97 : Activities of households as employers of domestic personnel,98 : Undifferentiated goods- and services-producing activities of private households for own use,99 : Activities of extraterritorial organisations and bodies,LSOA11CD,LSOA11NM,total_employment,log_total_employment
0,gor:London,1250,1250,400,0,1500,0,450,350,32000,...,57000,17000,62000,0,0,0,NaN,NaN,5605320.0,15.539227
1,lsoa2011:E01000907 : Camden 001A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,115.0,4.744932
2,lsoa2011:E01000908 : Camden 001B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,180.0,5.192957
3,lsoa2011:E01000909 : Camden 001C,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,NaN,NaN,200.0,5.298317
4,lsoa2011:E01000912 : Camden 001D,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,NaN,NaN,825.0,6.715383


In [34]:
# Get rid of value set on copy of slice warning
warnings.filterwarnings('ignore')

# Create Total Employment Column for Birmingham
empl_data_bham[empl_data_bham.columns[1:-2]] = empl_data_bham[empl_data_bham.columns[1:-2]].apply(pd.to_numeric, errors='coerce')
empl_data_bham['total_employment'] = empl_data_bham[empl_data_bham.columns[1:-2]].sum(axis=1)
empl_data_bham['log_total_employment'] = np.log(empl_data_bham['total_employment'].replace(0, np.nan))

empl_data_bham.head()

,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,08 : Other mining and quarrying,09 : Mining support service activities,10 : Manufacture of food products,...,94 : Activities of membership organisations,95 : Repair of computers and personal and household goods,96 : Other personal service activities,97 : Activities of households as employers of domestic personnel,98 : Undifferentiated goods- and services-producing activities of private households for own use,99 : Activities of extraterritorial organisations and bodies,LSOA11CD,LSOA11NM,total_employment,log_total_employment
13485,lsoa2011:E01008881 : Birmingham 067A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,E01008881,Birmingham 067A,455.0,6.120297
13486,lsoa2011:E01008882 : Birmingham 066A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,E01008882,Birmingham 066A,30.0,3.401197
13487,lsoa2011:E01008883 : Birmingham 078A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,...,20.0,0.0,0.0,0.0,0.0,0.0,E01008883,Birmingham 078A,1810.0,7.501082
13488,lsoa2011:E01008884 : Birmingham 078B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,...,0.0,5.0,40.0,0.0,0.0,0.0,E01008884,Birmingham 078B,2590.0,7.859413
13489,lsoa2011:E01008885 : Birmingham 076A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.0,0.0,0.0,0.0,E01008885,Birmingham 076A,90.0,4.499810


## LSOA Geographic Data

In [37]:
#Get LSOA Shapefile Data
lsoa_geo = gpd.read_file('data/lsoa_data/LSOA_2011_EW_BFE_V3.shp')

#Convert to WGS for consistency
lsoa_geo = lsoa_geo.to_crs(epsg=4326)

print("Num Rows (LSOAs): " + str(lsoa_geo.shape[0]))

#Get rid of columns I'm not using for now
lsoa_geo = lsoa_geo.drop(columns=['BNG_E', 'BNG_N', 'LONG_', 'LAT', 'GlobalID', 'Shape_Leng'])

#Join with population
lsoa_pop = pd.read_csv('data/lsoa_data/lsoa_pop.csv')

lsoa_geo.drop(columns=['LSOA11NM'])

#Get population estimates - had to be pulled in from separate Census dataset
lsoa_geo = lsoa_geo.merge(lsoa_pop, on='LSOA11CD')

lsoa_geo.head()

Num Rows (LSOAs): 34753


,LSOA11CD,LSOA11NM_x,geometry,LSOA11NM_y,Unnamed: 2,population
0,E01000034,Barking and Dagenham 003A,"POLYGON ((0.17380 51.56013, 0.17379 51.56012, ...",Barking and Dagenham 003A,NaN,"1,444"
1,E01000035,Barking and Dagenham 010A,"POLYGON ((0.16598 51.55250, 0.16601 51.55248, ...",Barking and Dagenham 010A,NaN,"1,610"
2,E01000036,Barking and Dagenham 010B,"POLYGON ((0.15965 51.54776, 0.15974 51.54771, ...",Barking and Dagenham 010B,NaN,"1,353"
3,E01000037,Barking and Dagenham 003B,"POLYGON ((0.18508 51.56480, 0.18476 51.56454, ...",Barking and Dagenham 003B,NaN,"1,688"
4,E01000038,Barking and Dagenham 003C,"POLYGON ((0.17599 51.56476, 0.17601 51.56475, ...",Barking and Dagenham 003C,NaN,"1,447"


## Overture Places Data

### London

In [40]:
london_places = gpd.read_file('data/overture_data/london_places.geojson')

print('Number of POIs in Overture (London): ' + str(london_places.shape[0]))
london_places.head()

Number of POIs in Overture (London): 343712


,name,category,confidence,geometry
0,The Heating Professionals,hvac_services,0.77,POINT (-0.50991 51.28709)
1,Hub Property Care,plumbing,0.77,POINT (-0.50991 51.28709)
2,F & S,plumbing,0.72,POINT (-0.50911 51.28766)
3,Hookins Accountants,accountant,0.75,POINT (-0.50881 51.28878)
4,Muddy Paws Grooming Service,pet_groomer,0.55,POINT (-0.50741 51.28859)


### Birmingham

In [41]:
bham_places = gpd.read_file('data/overture_data/bham_places.geojson')

print('Number of POIs in Overture (Birmingham): ' + str(bham_places.shape[0]))
bham_places.head()

Number of POIs in Overture (Birmingham): 46781


,name,category,confidence,geometry
0,Beacon Hill Toposcope,landmark_and_historical_building,0.96,POINT (-2.01965 52.38213)
1,Lickey Hills Country Park - Beacon Hill,park,0.67,POINT (-2.01972 52.38229)
2,Lickey Monument,monument,0.55,POINT (-2.01956 52.38227)
3,Beacon Castle,park,0.67,POINT (-2.01961 52.38234)
4,Barnett Fair Unisex Hairdressing,beauty_salon,0.96,POINT (-2.00227 52.38278)


### OpenStreetMap Buildings Data

I will query each set of building types and then collapse the dataset down so that tags are preserved

### London

### Birmingham